In [4]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import numpy as np

In [5]:
username="YOUR_ID" # Not required
#scope="user-read-recently-played"
ClientID="CLIENT_ID" # Get this from spotify developer portal
ClientSecret = "CLIENT_SECRET" # Get this from spotify developer portal

In [6]:
client_credentials_manager = SpotifyClientCredentials(client_id=ClientID, client_secret=ClientSecret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [7]:
playlists = sp.user_playlists('spotify')

In [10]:
idx = 0
print(f"Playlist {idx}: {playlists['items'][idx]['name']}")
playlist_id = playlists['items'][0]['id']

tracks_in_playlist = sp.user_playlist_tracks("spotify", playlist_id)

Playlist 0: Today's Top Hits


In [11]:
single_song = tracks_in_playlist['items'][-1]
song_info = single_song['track']

print(f"Info keys:")
for idx, key in enumerate(song_info.keys()):
    print(f"{idx:2d}: {key}")
print(f"Duration (milliseconds) {song_info['duration_ms']}")

Info keys:
 0: album
 1: artists
 2: available_markets
 3: disc_number
 4: duration_ms
 5: episode
 6: explicit
 7: external_ids
 8: external_urls
 9: href
10: id
11: is_local
12: name
13: popularity
14: preview_url
15: track
16: track_number
17: type
18: uri
Duration (milliseconds) 163804


In [12]:
print(f"Song name: {single_song['track']['name']}")
# Notes:
# Save: Track name, Artists (list), In Australia, song_id, 
# Trackname: track->name
# Artists: track->album->arists
# In Australia: "AU" in track->available_markets
# Song id: track->id
# duration: track->duration_ms
# explicit: track->explicit
# popularity: track->popularity
# type: track->type

def get_single_info(single_song_json):
    
    song_info = single_song_json['track']
    track_name = song_info['name']
    artists = [i['name'] for i in song_info['album']['artists']]
    in_aus = 'AU' in song_info['available_markets']
    song_id = song_info['id']
    duration = song_info['duration_ms']
    explicit = song_info['explicit']
    popularity = song_info['popularity']
    _type = song_info['type']
    album_id = song_info['album']['id'] 
    
    combined_info = [track_name, artists, in_aus, song_id, album_id, duration, explicit, popularity, _type]
    return combined_info

headers = ["track_name", "artists", "in_australia", "song_id", "album_id", "duration (ms)", "explicit", "song_popularity", "song_type"]
# Song audio analysis

Song name: Summer Days (feat. Macklemore & Patrick Stump of Fall Out Boy)


In [13]:
info = [get_single_info(i) for i in tracks_in_playlist['items']]

In [14]:
# To get from the album info
# 1. Copyright info (text + type)
# 2. Label
# 3. Album popularity
# 4. Album name
def get_album_info(single_album_json):

    copyrights = single_album_json['copyrights']
    copyrights_flag = [i['type'] for i in copyrights]
    copyrights_text = [i['text'] for i in copyrights]
    label = single_album_json['label']
    album_popularity = single_album_json['popularity']
    album_name = single_album_json['name']
    
    combined_info = [album_name, label, album_popularity, copyrights_text, copyrights_flag]
    return combined_info

album_headers = ["album_name", "label", "album_popularity", "copyrights_text", "copyrights_flag"]

In [88]:
album_ids = [i[4] for i in info]
album_info = []
for i in range(int(np.ceil(len(album_ids)/20))):
    # spotify album pull can only do a max of 20 at a time
    album_json = sp.albums(album_ids[20*i:20*(i+1)])
    for j in album_json['albums']:
        album_info.append(get_album_info(j))


In [107]:
df_album = pd.DataFrame(album_info, columns = album_headers)
df_song = pd.DataFrame(info, columns=headers)
playlist_df = pd.merge(df_song, df_album, left_index=True, right_index=True)

In [108]:
playlist_df.shape

(50, 14)

In [185]:
playlists = sp.user_playlists('spotify')
# for i in range(len(playlists['items'])):
#     playlists_id = playlists['items'][i]['id']
#     tracks_in_playlist = sp.user_playlist_tracks("spotify", playlist_id)
#     print(tracks_in_playlist['next'])
    


In [186]:
playlists['items'][0]

{'collaborative': False,
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DXcBWIGoYBM5M'},
 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DXcBWIGoYBM5M',
 'id': '37i9dQZF1DXcBWIGoYBM5M',
 'images': [{'height': None,
   'url': 'https://pl.scdn.co/images/pl/default/3d325546b685e9e38ac2dcf6160c774e7b1e6fde',
   'width': None}],
 'name': "Today's Top Hits",
 'owner': {'display_name': 'Spotify',
  'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
  'href': 'https://api.spotify.com/v1/users/spotify',
  'id': 'spotify',
  'type': 'user',
  'uri': 'spotify:user:spotify'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MTU2NTYyODM1OCwwMDAwMDM0ZDAwMDAwMTZjODZiOGRlZDIwMDAwMDE2YzczN2VkMzRi',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DXcBWIGoYBM5M/tracks',
  'total': 51},
 'type': 'playlist',
 'uri': 'spotify:playlist:37i9dQZF1DXcBWIGoYBM5M'}

Scrape all the data from the Spotify playlist main page. The following section would infinitely loop, so it is hard capped at 500 playlists

In [180]:
output_dfs = []
playlists = sp.user_playlists('spotify')
while playlists:
    for idx, playlist in enumerate(playlists['items']):
        
        playlist_id = playlist['id']
        print(f"Playlist {idx + 1 + playlists['offset']}: {playlist['name']} (id {playlist_id})")
        
        tracks_in_playlist = sp.user_playlist_tracks("spotify", playlist_id)
        
        # Wrap this in a try/except and iterate over each one to avoid some nasty problems
        # (that is keyerrors due to a bad response)
        #info = [get_single_info(i) for i in tracks_in_playlist['items']]
        info = []
        for item in tracks_in_playlist['items']:
            try:
                single = get_single_info(item)
                info.append(single)
            except:
                print(f"Failure to get single info for {item}")
        
        album_ids = [i[4] for i in info]
        album_info = []
        for i in range(int(np.ceil(len(album_ids)/20))):
            # spotify album pull can only do a max of 20 at a time
            album_json = sp.albums(album_ids[20*i:20*(i+1)])
            for j in album_json['albums']:
                album_info.append(get_album_info(j))

        df_album = pd.DataFrame(album_info, columns = album_headers)
        df_song = pd.DataFrame(info, columns=headers)
        playlist_df = pd.merge(df_song, df_album, left_index=True, right_index=True)

        playlist_df['playlist_name'] = playlist['name']

        output_dfs.append(playlist_df)
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None
        
    if idx + 1 + playlists['offset'] > 500:
        break

Playlist 1: Today's Top Hits (id 37i9dQZF1DXcBWIGoYBM5M)
Playlist 2: RapCaviar (id 37i9dQZF1DX0XUsuxWHRQd)
Playlist 3: Hot Country (id 37i9dQZF1DX1lVhptIYRda)
Playlist 4: ¡Viva Latino! (id 37i9dQZF1DX10zKzsJ2jva)
Playlist 5: New Music Friday (id 37i9dQZF1DX4JAvHpjipBk)
Playlist 6: mint (id 37i9dQZF1DX4dyzvuaRJ0n)
Playlist 7: Are & Be (id 37i9dQZF1DX4SBhb3fqCJd)
Playlist 8: Rock Classics (id 37i9dQZF1DWXRqgorJj26U)
Playlist 9: Peaceful Piano (id 37i9dQZF1DX4sWSpwq3LiO)
Playlist 10: Rock This (id 37i9dQZF1DXcF6B6QPhFDv)
Playlist 11: New Music Friday (id 37i9dQZF1DWXJfnUiYjUKT)
Playlist 12: TGIF (id 37i9dQZF1DXcRXFNfZr7Tp)
Playlist 13: All Out 00s (id 37i9dQZF1DX4o1oenSJRJd)
Playlist 14: All Out 90s (id 37i9dQZF1DXbTxeAdrVG2l)
Playlist 15: All Out 80s (id 37i9dQZF1DX4UtSsGT1Sbe)
Playlist 16: All Out 70s (id 37i9dQZF1DWTJ7xPn4vNaz)
Playlist 17: All Out 60s (id 37i9dQZF1DXaKIA8E7WcJj)
Playlist 18: All Out 50s (id 37i9dQZF1DWSV3Tk4GO2fq)
Playlist 19: Soft Pop Hits (id 37i9dQZF1DWTwnEm1IYyoj)

AttributeError: 'NoneType' object has no attribute 'split'

Now that we have the data, we can put it all together into a single large dataframe

In [128]:
from functools import reduce

In [170]:
big_df = reduce(lambda x, y: x.append(y), output_dfs)

In [143]:
big_df.to_csv("data.csv", index=False)

In [18]:
big_df.shape

NameError: name 'big_df' is not defined